# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
#! pip install gmaps

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
file = pd.read_csv("C:/Users/Owner/Desktop/Python-api-Challenge/output_data/cities.csv")
#file.dropna()
file.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind
0,Pyaozerskiy,68,RU,1610505646,91,65.8333,31.1667,-10.52,5.48
1,Ilulissat,20,GL,1610505646,86,69.2167,-51.1000,19.40,8.05
2,Quatre Cocos,75,MU,1610505646,88,-20.2078,57.7625,77.00,12.66
3,Mumford,0,GH,1610505646,86,5.2618,-0.7590,79.03,12.19
4,Hong Kong,0,HK,1610505412,30,22.2855,114.1577,57.99,5.57


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"]

In [5]:
# Heatmap of humidity
# Plot Heatmap
fig = gmaps.figure(center = [0,0], zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria
- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = file.loc[(file["Temp"] < 80) & (file["Temp"] > 70)
                                    & (file["Wind"] < 10)
                                    & (file["Cloudiness"] == 0)].dropna()
ideal_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind
5,Cape Town,0,ZA,1610505636,78,-33.9258,18.4232,72.00,6.91
71,San Rafael,0,AR,1610505656,32,-34.6177,-68.3301,71.83,7.34
103,Santa Fe,0,AR,1610505447,64,-31.6333,-60.7000,73.99,8.37
105,Nioro,0,GM,1610505661,23,13.3500,-15.7500,70.54,8.63
149,Wukari,0,NG,1610505668,23,7.8500,9.7833,70.07,4.92
161,Leer,0,SS,1610505669,27,8.3018,30.1418,70.25,6.04
214,Daoukro,0,CI,1610505676,95,7.0591,-3.9631,71.56,2.04
253,Bawku,0,GH,1610505680,19,11.0616,-0.2417,71.62,8.77
264,Calvinia,0,ZA,1610505682,61,-31.4707,19.7760,70.77,4.43


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
#hotel_df = ideal_weather.reset_index(drop=True)
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
5,Cape Town,ZA,-33.9258,18.4232,
71,San Rafael,AR,-34.6177,-68.3301,
103,Santa Fe,AR,-31.6333,-60.7000,
105,Nioro,GM,13.3500,-15.7500,
149,Wukari,NG,7.8500,9.7833,
161,Leer,SS,8.3018,30.1418,
214,Daoukro,CI,7.0591,-3.9631,
253,Bawku,GH,11.0616,-0.2417,
264,Calvinia,ZA,-31.4707,19.7760,


In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 50000,
    "types": "lodging",
    #"keyword": "hotels",
    "key": g_key
}

# Iterate through 
# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
5,Cape Town,ZA,-33.9258,18.4232,
71,San Rafael,AR,-34.6177,-68.3301,
103,Santa Fe,AR,-31.6333,-60.7000,
105,Nioro,GM,13.3500,-15.7500,
149,Wukari,NG,7.8500,9.7833,
161,Leer,SS,8.3018,30.1418,
214,Daoukro,CI,7.0591,-3.9631,
253,Bawku,GH,11.0616,-0.2417,
264,Calvinia,ZA,-31.4707,19.7760,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, 
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))